In [1]:
import balltracking.balltrack as blt
import os, glob
import fitsio
from scipy.ndimage.filters import gaussian_filter 
from scipy.signal import convolve2d
from scipy.io import readsav
import numpy as np
import matplotlib
#matplotlib.use('tkagg')
import matplotlib.pyplot as plt
from matplotlib import rc
# Load FLCT
from pathlib import Path, PurePath
from IPython.display import display
import time
import pandas as pd


matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'xtick.labelsize': 12, 'ytick.labelsize': 12})
matplotlib.rcParams.update({'axes.labelsize': 12})
pd.options.display.max_columns = None

In [29]:
def load_vel_mean(v_files, trange):
    "Load the velocity files and average over a time range"
    vxs = []
    vys = []
    vx_files_subset = v_files[0][trange[0]:trange[1]]
    vy_files_subset = v_files[1][trange[0]:trange[1]]
    for vxf, vyf in zip(vx_files_subset, vy_files_subset):
        vxs.append(fitsio.read(vxf))
        vys.append(fitsio.read(vyf))
    # Get the mean of the velocity components
    vx = np.array(vxs).mean(axis=0)
    vy = np.array(vys).mean(axis=0)
    return np.array([vx, vy])


def smooth_vel(v, fwhm, kernel='boxcar'):
    """ Smooth the velocity with a smoothing kernel that can either be: 
     - boxcar: width set to fwhm
     - gaussian: parametrized by fwhm. 
     
     Returns the smoothed velocity components
    """ 
    
    if kernel == 'boxcar':
        box = np.ones([fwhm, fwhm]) / fwhm**2
        vxs2 = convolve2d(v[0,...], box, mode='same')
        vys2 = convolve2d(v[1,...], box, mode='same')
    elif kernel == 'gaussian':
        sigma = fwhm / 2.35
        vxs2 = gaussian_filter(v[0,...], sigma=sigma, order=0)
        vys2 = gaussian_filter(v[1,...], sigma=sigma, order=0)
        
    vm = np.sqrt(vxs2**2 + vys2**2)
    return np.array([vxs2, vys2, vm])


# For FLCT
def read_vi(file, data_str):
    idl_dict = readsav(file)
    vi = idl_dict[data_str]
    return vi


def calc_c_pearson(vx1, vx2, vy1, vy2, fov=None):
    vx1f, vx2f, vy1f, vy2f = vx1[fov], vx2[fov], vy1[fov], vy2[fov]
    c_pearson = np.sum(vx1f*vx2f + vy1f*vy2f) / np.sqrt(np.sum(vx1f**2 + vy1f**2)*np.sum(vx2f**2 + vy2f**2))
    return c_pearson


In [3]:
datadir = os.path.join(os.environ['DATA'], 'Ben/SteinSDO')
outputdir = os.path.join(datadir, 'comparisons')

u = 368000 / 60

fwhms = [7,11,15]
kernels = ['boxcar', 'gaussian']
kernel = 'boxcar'
dims = [263, 263]
# Select a subfield excluding edge effects
# Use same padding size as in Tremblay et al. 2018 used pad = 10
fovs = [np.s_[fwhm:-fwhm:fwhm, fwhm:-fwhm:fwhm] for fwhm in fwhms]

nframes = range(30, 365, 5)
tranges = [[0, nt] for nt in range(30, 365, 5)]
nflows = len(tranges)
print("number of time-averaged flow maps: ", nflows)

number of time-averaged flow maps:  67


### Load Stein simulation data

In [4]:
# Load Stein data
svx_files = sorted(glob.glob(os.path.join(datadir,'SDO_vx*.fits')))
svy_files = sorted(glob.glob(os.path.join(datadir,'SDO_vy*.fits')))
#vx, vy = load_vel_mean((svx_files, svy_files), tranges[0])
v_stein_l = [load_vel_mean((svx_files, svy_files), trange) for trange in tranges]
# smooth the Stein velocities 
v_stein_sm = [[smooth_vel(v, fwhm, kernel=kernel) for v in v_stein_l] for fwhm in fwhms]
npts_ = [v_stein_sm[f][0][0][fov].size for f, fov in enumerate(fovs)]

### Balltracking data

In [58]:
caldf = pd.read_csv(os.path.expanduser('~/Data/sanity_check/stein_series/correlation_dataframe.csv'))
caldf.head()

,rs,intsteps,ballspacing,dp,sigma_factor,fourier_radius,kernel,fwhm,corr_uncal,corr,corr_top,corr_bot,MAE_uncal_vx,MAE_uncal_vy,MAE_cal_vx,MAE_cal_vy,RMSE_uncal_vx,RMSE_uncal_vy,RMSE_cal_vx,RMSE_cal_vy,p_top_0,p_top_1,p_bot_0,p_bot_1,vx_top -0.20,vx_top -0.16,vx_top -0.12,vx_top -0.08,vx_top -0.04,vx_top 0.00,vx_top 0.04,vx_top 0.08,vx_top 0.12,vx_top 0.16,vx_top 0.20,vx_bottom -0.20,vx_bottom -0.16,vx_bottom -0.12,vx_bottom -0.08,vx_bottom -0.04,vx_bottom 0.00,vx_bottom 0.04,vx_bottom 0.08,vx_bottom 0.12,vx_bottom 0.16,vx_bottom 0.20,MAE_uncal_vx_top,MAE_uncal_vx_bot,MAE_cal_vx_top,MAE_cal_vx_bot
0,2,3,1,0.2,1.0,0,boxcar,7,0.749145,0.747199,0.700015,0.646444,415.294364,424.489917,354.143782,335.334280,521.646319,524.663171,439.313876,422.866456,3.901554,0.010838,4.435106,0.010395,-0.054124,-0.044101,-0.033369,-0.022909,-0.012670,-0.002936,0.007171,0.017497,0.028227,0.038062,0.048595,-0.047259,-0.038889,-0.029210,-0.020261,-0.011464,-0.001900,0.006592,0.015412,0.024779,0.033327,0.043092,397.345953,442.537769,397.345953,442.537769
1,2,3,1,0.2,1.0,1,boxcar,7,0.875152,0.874690,0.830433,0.815424,418.438780,427.792064,242.725194,237.449878,522.199136,530.809972,300.763124,301.193551,4.547713,0.013899,5.469452,0.008893,-0.047306,-0.038455,-0.029303,-0.020395,-0.011576,-0.003212,0.006275,0.014259,0.023494,0.031499,0.041099,-0.038371,-0.030785,-0.023593,-0.016240,-0.008708,-0.001320,0.005307,0.013004,0.020097,0.027776,0.034948,275.093665,299.724301,275.093665,299.724301
2,2,3,1,0.2,1.0,2,boxcar,7,0.872427,0.871615,0.829833,0.806353,416.270739,426.494909,240.826499,242.669774,519.681158,529.280115,301.876954,307.650007,4.519459,0.013772,5.360508,0.009861,-0.047190,-0.038802,-0.029717,-0.020448,-0.011907,-0.003005,0.006280,0.014376,0.023532,0.032056,0.041307,-0.039537,-0.031579,-0.024329,-0.016336,-0.009166,-0.001709,0.005600,0.013109,0.020247,0.027952,0.035512,277.276603,301.956416,277.276603,301.956416
3,2,3,1,0.2,1.0,3,boxcar,7,0.868528,0.867519,0.827131,0.797443,415.195661,423.107641,245.663652,246.526164,519.660821,526.760048,309.046103,309.114655,4.408601,0.013569,5.175517,0.009297,-0.048534,-0.039410,-0.030309,-0.020969,-0.012361,-0.003097,0.006283,0.015185,0.023978,0.032992,0.042386,-0.041187,-0.032664,-0.024405,-0.016932,-0.009317,-0.001654,0.005751,0.013360,0.021501,0.028905,0.036882,285.845471,312.383094,285.845471,312.383094
4,2,3,1,0.2,1.0,4,boxcar,7,0.857986,0.857253,0.810646,0.788075,413.388203,422.217178,260.874174,252.341002,517.179536,522.892785,326.481800,314.983899,4.276099,0.012365,4.947788,0.010256,-0.050042,-0.040041,-0.031004,-0.021256,-0.012114,-0.003239,0.006659,0.015780,0.025089,0.034388,0.043973,-0.042526,-0.034738,-0.025835,-0.018485,-0.010096,-0.001878,0.006012,0.014144,0.022106,0.030023,0.038472,296.784477,329.703579,296.784477,329.703579


In [67]:
cols = ['rs', 'intsteps', 'dp', 'sigma_factor', 'fourier_radius', 'kernel', 'corr_uncal', 'corr', 'corr_top', 'corr_bot', 'MAE_uncal_vx', 'MAE_cal_vx', 'MAE_cal_vx_top',  'MAE_cal_vx_bot', 'MAE_uncal_vx_top', 'MAE_uncal_vx_bot', 'RMSE_uncal_vx', 'RMSE_cal_vx', 'p_top_0', 'p_bot_0']
idx_cols = [caldf.columns.get_loc(c) for idx, c in enumerate(cols)]
caldf.iloc[[np.argmax(caldf['corr'].values), np.argmax(caldf['corr_top'].values), np.argmax(caldf['corr_bot'].values), np.argmin(caldf['MAE_cal_vx'].values), np.argmin(caldf['MAE_cal_vx_top'].values), np.argmin(caldf['MAE_cal_vx_bot'].values)], idx_cols]
           

,rs,intsteps,dp,sigma_factor,fourier_radius,kernel,corr_uncal,corr,corr_top,corr_bot,MAE_uncal_vx,MAE_cal_vx,MAE_cal_vx_top,MAE_cal_vx_bot,MAE_uncal_vx_top,MAE_uncal_vx_bot,RMSE_uncal_vx,RMSE_cal_vx,p_top_0,p_bot_0
3021,2,5,0.25,1.50,1,boxcar,0.902267,0.902203,0.868297,0.874885,282.245967,209.935932,244.116172,240.807274,244.116172,240.807274,356.015522,264.428362,1.801228,1.713018
2492,2,5,0.25,2.00,2,boxcar,0.894176,0.893239,0.868541,0.839359,383.617761,226.604721,247.910467,279.789857,247.910467,279.789857,479.696996,280.846312,3.358044,3.857307
1842,2,4,0.23,2.00,2,boxcar,0.901524,0.901283,0.860641,0.879542,304.482870,210.965764,248.810163,232.058504,248.810163,232.058504,381.294669,265.461928,2.068565,1.957719
2881,2,5,0.22,1.75,1,boxcar,0.901054,0.900901,0.862196,0.877141,293.057684,207.154935,243.864676,234.969570,243.864676,234.969570,366.644237,260.694727,1.887281,1.798932
1962,2,4,0.26,1.25,2,boxcar,0.898035,0.898009,0.863623,0.868165,289.733881,211.682389,242.613414,247.538064,242.613414,247.538064,364.393887,266.348979,1.857790,1.779792
1842,2,4,0.23,2.00,2,boxcar,0.901524,0.901283,0.860641,0.879542,304.482870,210.965764,248.810163,232.058504,248.810163,232.058504,381.294669,265.461928,2.068565,1.957719


In [ ]:
cal_file2 = os.path.expanduser('~/Data/sanity_check/stein_series/calibration/calibration_fwhm_kernel.csv')
caldf2 = pd.read_csv(cal_file2)
display(caldf2.head())

In [ ]:

dfs1 = []
dfs2 = []

for i, fwhm_i in enumerate(fwhms):
    fov = fovs[i]
    # Load Balltrack velocities. processed with balltracking_scripts/generic_balltrack.py, calibrated with balltracking/generic_calibration
    # bvxs = [np.load(os.path.join(datadir, 'balltrack/vxy_{:s}_fwhm_{:d}_avg_{:d}.npz'.format(kernel, fwhm, trange[1])))['vx'] for trange in tranges]
    bvs = [np.array([np.load(os.path.join(datadir, 'balltrack/vxy_{:s}_fwhm_{:d}_avg_{:d}.npz'.format(kernel, fwhm_i, trange[1])))[var] for var in ['vx', 'vy']]) for trange in tranges]
    bvsm = [np.sqrt(bvs[j][0][:]**2 + bvs[j][1][:]**2) for j in range(nflows)]
    
    # Fit Balltracking to Stein simulation
    bps, brs, _, _, _ = zip(*[np.polyfit(bvs[j][0][fov].ravel()*u, v_stein_sm[i][j][0][fov].ravel(), 1, full=True) for j in range(nflows)])
    brs = [np.sqrt(br[0]/npts_[i]) for br in brs]
    print('Balltrack residuals from fit: ', brs[0])
    b_alphas, b_offsets = map(list, zip(*bps))
    # Calculate residuals RMSE
    error_field = [np.abs(v_stein_sm[i][j][0][fov] - bvs[j][0][fov]*u).ravel() for j in range(nflows)]
    RMSEs = [np.sqrt(np.mean(error_field[j]**2)) for j in range(nflows)]
    # print('RMSE Balltracking = ',  b_RMSEs[0])
    # Relative error (Mean Absolute Percentage Error: MAPE)
    APE_field_masked = [np.abs((v_stein_sm[i][j][2][fov] - bvsm[j][fov]*u)/np.ma.masked_less_equal(v_stein_sm[i][j][2][fov], 10)).ravel()*100 for j in range(nflows)]
    df1 = pd.concat([pd.DataFrame({'vx_stein': v_stein_sm[i][j][0][fov].ravel(), 'vy_stein': v_stein_sm[i][j][1][fov].ravel(),
                                   'vx': bvs[j][0][fov].ravel()*u, 'vy': bvs[j][1][fov].ravel()*u, 'vmag':bvsm[j][fov].ravel()*u,
                                   'vmag_stein': v_stein_sm[i][j][2][fov].ravel(), 'error':error_field[j], 'APE': APE_field_masked[j], 
                                   'nframes':nframes[j], 'FWHM':fwhm_i, 'kernel':kernel, 'Algorithm':'Balltracking'}) for j in range(nflows)])
    
    APE_field = [np.abs((v_stein_sm[i][j][2][fov] - bvsm[j][fov]*u)/v_stein_sm[i][j][2][fov]).ravel()*100 for j in range(nflows)]
    bMAPE = [np.median(APE_field[j]) for j in range(nflows)]
    # Correlations between Balltracking and simulation
    bcorrs = [calc_c_pearson(v_stein_sm[i][j][0], bvs[j][0], v_stein_sm[i][j][1], bvs[j][1], fov=fov) for j in range(len(tranges))]
    df2 = pd.DataFrame({'nframes':nframes, 'slope':b_alphas, 'RMSE':RMSEs, 'MAPE':bMAPE, 'correlation':bcorrs, 'FWHM':fwhm_i, 'kernel':kernel, 'Algorithm':'Balltracking'})
    
    dfs1.append(df1)
    dfs2.append(df2)

bdf1 = pd.concat(dfs1)
bdf2 = pd.concat(dfs2)
display(bdf1.head())
display(bdf2.head())

### FLCT data

In [ ]:
# Load fits files
datadir_flct = os.path.expanduser('~/Data/Ben/SteinSDO/FLCT_Raphael/output_FLCT_sigma4')
fvx_files = sorted((glob.glob(os.path.join(datadir_flct, 'FLCT_vx*.fits'))))
fvy_files = sorted((glob.glob(os.path.join(datadir_flct, 'FLCT_vy*.fits'))))

In [ ]:
# Average over time
fvs = [load_vel_mean((fvx_files, fvy_files), trange) for trange in tranges]

In [ ]:
dfs1 = []
dfs2 = []

for i, fwhm_i in enumerate(fwhms):
    if fwhm_i == 7:
        fvs2 = [np.array([fv[0], fv[1], np.sqrt(fv[0][:]**2 + fv[1][:]**2)]) for fv in fvs]
    else:
        # Smooth Stein velocities
        fvs2 = [smooth_vel(fv, fwhm_i, kernel=kernel) for fv in fvs]
    
    fov = fovs[i]
    # Fit FLCT to Stein simulation
    fps, frs, _, _, _ = zip(*[np.polyfit(fvs2[j][0][fov].ravel()*u, v_stein_sm[i][j][0][fov].ravel(), 1, full=True) for j in range(nflows)])
    frs = [np.sqrt(fr[0]/npts_[i]) for fr in frs]
    print('FLCT residuals from fit: ', frs[0])
    f_alphas, f_offsets = map(list, zip(*fps))
    # Calculate residuals RMSE
    error_field = [np.abs(v_stein_sm[i][j][0][fov] - fvs2[j][0][fov]*u).ravel() for j in range(nflows)]
    RMSE_uncal = [np.sqrt(np.mean(error_field[j]**2)) for j in range(nflows)]
    # print('RMSE FLCT (uncalibrated) = ',  fRMSE_uncals[0])
    # Relative error (Mean Absolute Percentage Error: MAPE)
    APE_field_masked = [np.abs((v_stein_sm[i][j][2][fov] - fvs2[j][2][fov]*u)/np.ma.masked_less_equal(v_stein_sm[i][j][2][fov], 10)).ravel()*100 for j in range(nflows)]
    df1 = pd.concat([pd.DataFrame({'vx_stein': v_stein_sm[i][j][0][fov].ravel(), 'vy_stein': v_stein_sm[i][j][1][fov].ravel(),
                                   'vx':fvs2[j][0][fov].ravel()*u, 'vy':fvs2[j][1][fov].ravel()*u, 'vmag':fvs2[j][2][fov].ravel()*u,
                                   'vmag_stein': v_stein_sm[i][j][2][fov].ravel(), 'error':error_field[j], 'APE': APE_field_masked[j], 
                                   'nframes':nframes[j], 'FWHM':fwhm_i, 'kernel':kernel, 'Algorithm':'FLCT'}) for j in range(nflows)])  
    
    APE_field = [np.abs((v_stein_sm[i][j][2][fov] - fvs2[j][2][fov]*u)/v_stein_sm[i][j][2][fov]).ravel()*100 for j in range(nflows)]
    fMAPE = [np.median(APE_field[j]) for j in range(nflows)]
    # Correlations between FLCT and simulation
    fcorrs = [calc_c_pearson(v_stein_sm[i][j][0], fvs2[j][0], v_stein_sm[i][j][1], fvs2[j][1], fov=fov) for j in range(nflows)]
    df2 = pd.DataFrame({'nframes':nframes, 'slope':f_alphas, 'RMSE':RMSE_uncal, 'MAPE':fMAPE, 'correlation':fcorrs, 'FWHM':fwhm_i, 'kernel':kernel, 'Algorithm':'FLCT'})
    dfs1.append(df1)
    dfs2.append(df2)
    
fdf1 = pd.concat(dfs1)
fdf2 = pd.concat(dfs2)
display(fdf1.head())
display(fdf2.head())

In [ ]:
df1 = pd.concat([bdf1, fdf1]).reset_index()
df2 = pd.concat([bdf2, fdf2]).reset_index()

In [ ]:
df2.query("Algorithm == 'FLCT' & FWHM == '15'").head()

In [ ]:
df2.query("Algorithm == 'Balltracking' & FWHM == '15'").head()

In [ ]:
import seaborn as sns
sns.set(font_scale=2)
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
g = sns.lmplot(data=df1.query("nframes == 190 & FWHM==7"), x='vmag', y='vmag_stein', hue='Algorithm', markers=["o", "x"], height=7, legend_out=False)
g.set(xlim=[0, 1200], ylim=[0, 1200], xlabel=r'$v_{\mathrm{sim}}$ [m/s]', ylabel='v [m/s]');

In [ ]:
g = sns.lmplot(data=df1.query("nframes == 190 & FWHM==7"), x='vx', y='vx_stein', hue='Algorithm', markers=["o", "x"], height=7, legend_out=False)
#g.set(xlim=[0, 1200], ylim=[0, 1200], xlabel=r'$v_{\mathrm{sim}}$ [m/s]', ylabel='v [m/s]')

In [ ]:
g = sns.lmplot(data=df1.query("nframes == 190 & FWHM==7"), x='vy', y='vy_stein', hue='Algorithm', markers=["o", "x"], height=7, legend_out=False)
#g.set(xlim=[0, 1200], ylim=[0, 1200], xlabel=r'$v_{\mathrm{sim}}$ [m/s]', ylabel='v [m/s]')

In [ ]:
df1.query("nframes == 190 & FWHM==7").vx_stein.values

In [ ]:
p, r, _, _, _ = np.polyfit(df1.query("Algorithm == 'Balltracking' & nframes == 30 & FWHM==11").vx_stein.values, df1.query("Algorithm == 'Balltracking' & nframes == 30 & FWHM==11").vx.values, 1, full=True)
p

In [ ]:
g1 = sns.relplot(data=df1, x='nframes', y='error',style='Algorithm', hue='Algorithm', col='FWHM', kind='line', ci='sd', legend='full', 
            palette='colorblind', height=8)
g1.set(xlabel='Time average [min]', ylabel='error [m/s]', ylim=[0, 350])
plt.savefig(PurePath(outputdir, 'error.png'))

In [ ]:
g1 = sns.relplot(data=df1, x='nframes', y='APE',style='Algorithm', hue='Algorithm', col='FWHM', kind='line', ci='sd', legend='full', 
            palette='colorblind', height=8, estimator=np.median)
g1.set(xlabel='Time average [min]', ylabel='MAPE [%]', ylim=[0, 100])
plt.savefig(PurePath(outputdir, 'MAPE.png'))

In [ ]:
g1 = sns.relplot(data=df2, x='nframes', y='correlation', style='Algorithm', col='kernel', hue='FWHM', size='FWHM', kind='line', legend='full', 
            palette='colorblind', height=8)
g1.set(xlabel='Time average [min]')
plt.savefig(PurePath(outputdir, 'correlation.png'))

In [ ]:
g2 = sns.relplot(data=df2, x='nframes', y='RMSE', style='Algorithm', col='kernel', hue='FWHM', size='FWHM', kind='line', legend='full', 
            palette='colorblind', height=8)
g2.set(xlabel='Time average [min]', ylabel='RMSE [m/s]', ylim=[0, 350])
plt.savefig(PurePath(outputdir, 'rmse.png'))


In [ ]:
g3 = sns.relplot(data=df2, x='nframes', y='MAPE', style='Algorithm', col='kernel', hue='FWHM', size='FWHM', kind='line', legend='full', 
            palette='colorblind', height=8)
g3.set(xlabel='Time average [min]', ylabel='MAPE [%]')
plt.savefig(PurePath(outputdir, 'mape.png'))

In [ ]:
def hexbin(x, y, color, **kwargs):
    #cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, mincnt=2, gridsize=(10, 30), cmap='inferno', **kwargs)
    # The number of total counts (i.e. # of independent samples) changes with the FWHM
    plt.colorbar();

In [ ]:
sns.set(font_scale=4)
with sns.axes_style("whitegrid"):
    g = sns.FacetGrid(df1.query("nframes == 190"), row='Algorithm', col='FWHM', height=15)
    
g.map(hexbin, 'vmag', 'APE');
g.set(xlim=[0, 1250], ylim=[0, 100]);
cbar_ax = g.fig.add_axes([1.015,0.13, 0.015, 0.8])

In [ ]:
g = sns.FacetGrid(df1.query("nframes == 190"), row='Algorithm', col='FWHM', height=15)
g.map(sns.distplot, 'APE', bins=50, hist_kws={"range":[0, 100]})
g.set(xlim=[0, 200])
# g.set(xlabel='APE [%]', ylabel = 'density')

for ax in g.axes.flat:
    #ax.set_ticks(ax.get_title(), fontsize='xx-large')
    # This only works for the left ylabels
    #ax.set_xticks(range(0,101, 10))
    _ = plt.setp(ax.get_xticklabels(), visible=True)
    
g.axes[0, 0].set_ylabel('Probability density')
g.axes[1, 0].set_ylabel('Probability density')

### Drift test